In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import csv
import time
import re
import urllib
import json
import numpy as np
import threading
from tqdm import tqdm
import pickle
import cloudscraper
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.by import By
from selenium_stealth import stealth

In [2]:
with open(f"../use_link_load_data_0_1000000/link_data_stack_overflow_python_0_200000.pickle", "rb") as f:
    save_link = pickle.load(f)

In [6]:
max_pages = len(save_link)
count_thread = 160
per_vm = 8
vm_number = 0 + 10*0
num_per_thread = max_pages//count_thread
page_last = max_pages%count_thread
all_thread = [None]*count_thread
code_question = [None]*count_thread
code_answer = [None]*count_thread
webs = "https://stackoverflow.com/"
hours = 10
minute = 30

In [7]:
def save_code_question_answer_all_data(code_question,code_answer,start,stop,number_thread):
    try:
        st = time.time()
        if code_question[number_thread] == None:
            count = 0
            code_answer_in_link = []
            code_question_in_link = []
        else:
            count = start%num_per_thread
            code_answer_in_link = code_answer[number_thread]
            code_question_in_link = code_question[number_thread]
        for i in save_link[start:stop]:
            url = "https://stackoverflow.com/" + i
            from selenium import webdriver 
            from selenium.webdriver.chrome.service import Service as ChromeService 
            from webdriver_manager.chrome import ChromeDriverManager 
            
            options = webdriver.ChromeOptions() #newly added 
            options.headless = True #newly added 
            while True:
                with webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options) as driver: #modified 
                    # stealth(driver,
                    # languages=["en-US", "en"],
                    # vendor="Google Inc.",
                    # platform="Win32",
                    # webgl_vendor="Intel Inc.",
                    # renderer="Intel Iris OpenGL Engine",
                    # fix_hairline=True,
                    # )
                    driver.get(url)
                    #print("Page URL:", driver.current_url) 
                    #print("Page Title:", driver.title)
                    answers = driver.find_elements(By.XPATH, "//div[@id='answers']//code")
                    questions = driver.find_elements(By.CLASS_NAME,'question-hyperlink')
                    #print(answers[0].text)
                    if (driver.title != "Just a moment...") and (len(questions) > 0):
                        answers_in_link = []
                        for j in answers:
                            answers_in_link.append(j.text)
                        code_answer_in_link.append(answers_in_link)
                        code_question_in_link.append(questions[0].text)
                        break
                    else:
                        pass
            count += 1
            if count%10 == 0:
                print(f"Thread{number_thread}:count : {count}")
                print("*"*20)
        code_answer[number_thread] = code_answer_in_link
        code_question[number_thread] = code_question_in_link
        print(f"thread : {number_thread} finish")
        print(time.time()-st,"sec")
        print("*"*20)
    except:
        # print(f"thread : {number_thread} error\n{err}")
        code_answer[number_thread] = code_answer_in_link
        code_question[number_thread] = code_question_in_link
        save_code_question_answer_all_data(code_question,code_answer,start+count,stop,number_thread)

In [8]:
li = (page_last - 1)
for i in range(vm_number*per_vm,(vm_number+1)*per_vm):
    if i < page_last:
        all_thread[i] = threading.Thread(target=save_code_question_answer_all_data,args=(code_question,code_answer,i*num_per_thread + i*(1),(i+1)*num_per_thread + (i+1)*1,i))
        print(i*num_per_thread + i*(1),(i+1)*num_per_thread + (i+1)*1,i)
    elif page_last == 0:
        all_thread[i] = threading.Thread(target=save_code_question_answer_all_data,args=(code_question,code_answer,i*num_per_thread,(i+1)*num_per_thread,i))
        print(i*num_per_thread,(i+1)*num_per_thread,i)
    else:
        all_thread[i] = threading.Thread(target=save_code_question_answer_all_data,args=(code_question,code_answer,i*num_per_thread + (li+1)*(1),(i+1)*num_per_thread + (li+1)*1,i))
        print(i*num_per_thread + (li+1)*(1),(i+1)*num_per_thread + (li+1)*1,i)

0 1250 0
1250 2500 1
2500 3750 2
3750 5000 3
5000 6250 4
6250 7500 5
7500 8750 6
8750 10000 7


In [9]:
for i in range(vm_number*per_vm,(vm_number+1)*per_vm):
    all_thread[i].start()
for i in range(vm_number*per_vm,(vm_number+1)*per_vm):
    all_thread[i].join()

Thread6:count : 10
********************
Thread4:count : 10
********************
Thread7:count : 10
********************
Thread5:count : 10
********************
Thread3:count : 10
********************
Thread2:count : 10
********************
Thread1:count : 10
********************
Thread3:count : 20
********************
Thread7:count : 20
********************
Thread5:count : 20
********************
Thread4:count : 20
********************
Thread2:count : 20
********************
Thread6:count : 20
********************
Thread1:count : 20
********************
Thread0:count : 10
********************
Thread3:count : 30
********************
Thread7:count : 30
********************
Thread6:count : 30
********************
Thread5:count : 30
********************
Thread4:count : 30
********************
Thread1:count : 30
********************
Thread2:count : 30
********************
Thread6:count : 40
********************
Thread0:count : 20
********************
Thread7:count : 40
********************
